In [1]:
!pip install openpyxl

In [2]:
!pip install statsmodels

In [3]:
!pip install xgboost

In [4]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import f_oneway

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import xgboost as xgb
from sklearn.metrics import r2_score, accuracy_score, classification_report, precision_recall_fscore_support

In [6]:
import pickle

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/credit risk modelling/merged_data.xlsx')

In [9]:
df_unseen = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/credit risk modelling/Unseen_Dataset.xlsx')

In [10]:
df.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,Total_TL_opened_L12M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,Auto_TL,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,num_times_delinquent,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,num_times_30p_dpd,num_times_60p_dpd,num_std,num_std_6mts,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_6mts,num_dbt_12mts,num_lss,num_lss_6mts,num_lss_12mts,recent_level_of_deliq,tot_enq,CC_enq,CC_enq_L6m,CC_enq_L12m,PL_enq,PL_enq_L6m,PL_enq_L12m,time_since_recent_enq,enq_L12m,enq_L6m,enq_L3m,MARITALSTATUS,EDUCATION,AGE,GENDER,NETMONTHLYINCOME,Time_With_Curr_Empr,pct_of_active_TLs_ever,pct_opened_TLs_L6m_of_L12m,pct_currentBal_all_TL,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,0,0,0.00,0.000,0,0,0,0,1,0,4,1,4,0,72,18,549,11,29,0,0,0,0,0,21,5,11,0,0,0,0,0,0,0,0,0,29,6,0,0,0,6,0,0,566,0,0,0,Married,12TH,48,M,51000,114,0.200,0.0,0.798,0,1,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,1,0,1.00,0.000,0,0,0,1,0,0,0,0,1,0,7,7,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,209,1,0,0,Single,GRADUATE,23,F,19000,50,1.000,0.0,0.370,0,0,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,2,0,0.25,0.000,1,1,0,6,1,0,0,2,6,0,47,2,302,9,25,1,9,8,0,0,10,5,10,0,0,0,0,0,0,0,0,0,25,4,0,0,0,0,0,0,587,0,0,0,Married,SSC,40,M,18,191,1.000,0.5,0.585,0,0,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.0,0.333,0.667,0,0,0.00,0.000,0,1,0,0,0,0,0,3,0,2,131,32,583,0,0,0,0,0,0,0,53,4,16,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3951,0,0,0,Married,POST-GRADUATE,48,M,15000,75,0.333,0.0,0.000,0,0,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.0,0.167,0.833,0,1,0.00,0.167,0,4,0,0,2,0,0,6,0,0,150,17,245,14,270,0,0,0,13,11,5,0,2,3,0,1,0,0,0,0,0,0,26,15,2,0,1,7,3,3,7,6,5,4,Married,12TH,35,M,0,154,0.167,0.0,0.000,0,0,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [ ]:
categorical_columns = []

for i in list(df.columns):
  if df[i].dtype == 'object':
    categorical_columns.append(i)

df[categorical_columns].head()

,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2
0,Married,12TH,M,PL,PL
1,Single,GRADUATE,F,ConsumerLoan,ConsumerLoan
2,Married,SSC,M,ConsumerLoan,others
3,Married,POST-GRADUATE,M,AL,AL
4,Married,12TH,M,ConsumerLoan,PL


In [ ]:
from scipy.stats import chi2_contingency

for i in categorical_columns[:-1]:
  chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
  print(i, '---', pval)

KeyError: 'Approved_Flag'

Since all the values are <= 0.05 we will accept all the categorical columns.

In [13]:
numerical_columns = []

for i in list(df.columns):
  if df[i].dtype != 'object' and i not in ['PROSPECTID', 'Approved_Flag']:
    numerical_columns.append(i)

print(numerical_columns)

['Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL', 'Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_tl_open_L6M', 'pct_tl_closed_L6M', 'pct_active_tl', 'pct_closed_tl', 'Total_TL_opened_L12M', 'Tot_TL_closed_L12M', 'pct_tl_open_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'Auto_TL', 'CC_TL', 'Consumer_TL', 'Gold_TL', 'Home_TL', 'PL_TL', 'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL', 'time_since_recent_payment', 'num_times_delinquent', 'max_recent_level_of_deliq', 'num_deliq_6mts', 'num_deliq_12mts', 'num_deliq_6_12mts', 'num_times_30p_dpd', 'num_times_60p_dpd', 'num_std', 'num_std_6mts', 'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt', 'num_dbt_6mts', 'num_dbt_12mts', 'num_lss', 'num_lss_6mts', 'num_lss_12mts', 'recent_level_of_deliq', 'tot_enq', 'CC_enq', 'CC_enq_L6m', 'CC_enq_L12m', 'PL_enq', 'PL_enq_L6m', 'PL_enq_L12m', 'time_since_recent_enq', 'enq_L12m', 'enq_L6m', 'enq_L3m', 'AGE', 'NETMONTHLYINCOME', 'Time_With_Curr_Empr', 'pct_

In [14]:
# Calculating the VIF
vif_data = df[numerical_columns]
total_columns = vif_data.shape[1]
columns_to_keep = []
column_index = 0

for i in range(0, total_columns):
  vif_value = variance_inflation_factor(vif_data, column_index)
  print(numerical_columns[i], '-----', vif_value)

  if vif_value <= 6:
    columns_to_keep.append(numerical_columns[i])
    column_index += 1

  else:
    vif_data = vif_data.drop([numerical_columns[i]], axis = 1)

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Total_TL ----- inf


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Tot_Closed_TL ----- inf
Tot_Active_TL ----- 11.320180023967996
Total_TL_opened_L6M ----- 8.363698035000336
Tot_TL_closed_L6M ----- 6.520647877790928
pct_tl_open_L6M ----- 5.149501618212625
pct_tl_closed_L6M ----- 2.611111040579735


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


pct_active_tl ----- inf
pct_closed_tl ----- 1788.7926256209232
Total_TL_opened_L12M ----- 8.601028256477228
Tot_TL_closed_L12M ----- 3.832800792153077
pct_tl_open_L12M ----- 6.099653381646723
pct_tl_closed_L12M ----- 5.581352009642766
Tot_Missed_Pmnt ----- 1.985584353098778


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Auto_TL ----- inf
CC_TL ----- 4.80953830281934
Consumer_TL ----- 23.270628983464636
Gold_TL ----- 30.595522588100053
Home_TL ----- 4.384346405965583
PL_TL ----- 3.0646584155234238
Secured_TL ----- 2.898639771299251
Unsecured_TL ----- 4.377876915347324
Other_TL ----- 2.207853583695844
Age_Oldest_TL ----- 4.916914200506864
Age_Newest_TL ----- 5.214702030064725
time_since_recent_payment ----- 3.3861625024231476
num_times_delinquent ----- 7.840583309478997
max_recent_level_of_deliq ----- 5.255034641721434


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


num_deliq_6mts ----- inf
num_deliq_12mts ----- 7.380634506427238
num_deliq_6_12mts ----- 1.4210050015175733
num_times_30p_dpd ----- 8.083255010190316
num_times_60p_dpd ----- 1.6241227524040114
num_std ----- 7.257811920140003
num_std_6mts ----- 15.59624383268298
num_std_12mts ----- 1.825857047132431
num_sub ----- 1.5080839450032664
num_sub_6mts ----- 2.172088834824578
num_sub_12mts ----- 2.6233975535272274
num_dbt ----- 2.2959970812106176
num_dbt_6mts ----- 7.360578319196446
num_dbt_12mts ----- 2.1602387773102567
num_lss ----- 2.8686288267891467
num_lss_6mts ----- 6.458218003637272
num_lss_12mts ----- 2.8474118865638247
recent_level_of_deliq ----- 4.753198156284083
tot_enq ----- 16.22735475594825
CC_enq ----- 6.424377256363877
CC_enq_L6m ----- 8.887080381808678
CC_enq_L12m ----- 2.3804746142952653
PL_enq ----- 8.60951347651454
PL_enq_L6m ----- 13.06755093547673
PL_enq_L12m ----- 3.500040056654653
time_since_recent_enq ----- 1.9087955874813773
enq_L12m ----- 17.006562234161628
enq_L6m --

In [15]:
vif_data.columns

Index(['pct_tl_open_L6M', 'pct_tl_closed_L6M', 'Tot_TL_closed_L12M',
       'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'CC_TL', 'Home_TL', 'PL_TL',
       'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL',
       'Age_Newest_TL', 'time_since_recent_payment',
       'max_recent_level_of_deliq', 'num_deliq_6_12mts', 'num_times_60p_dpd',
       'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt',
       'num_dbt_12mts', 'num_lss', 'num_lss_12mts', 'recent_level_of_deliq',
       'CC_enq_L12m', 'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m',
       'NETMONTHLYINCOME', 'Time_With_Curr_Empr', 'pct_currentBal_all_TL',
       'CC_Flag', 'PL_Flag', 'pct_PL_enq_L6m_of_ever',
       'pct_CC_enq_L6m_of_ever', 'HL_Flag', 'GL_Flag'],
      dtype='object')

In [16]:
#check Anova for columns_to_be_kept

columns_to_be_kept_numerical = []

for i in columns_to_keep:
  a = list(df[i])
  b = list(df['Approved_Flag'])

  group_P1 = [value for value, group in zip(a,b) if group =='P1']
  group_P2 = [value for value, group in zip(a,b) if group =='P2']
  group_P3 = [value for value, group in zip(a,b) if group =='P3']
  group_P4 = [value for value, group in zip(a,b) if group =='P4']

  f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

  print(i, "----", p_value)
  if p_value <= 0.05:
    columns_to_be_kept_numerical.append(i)

pct_tl_open_L6M ---- 0.0
pct_tl_closed_L6M ---- 2.676086811190998e-52
Tot_TL_closed_L12M ---- 1.762685578230421e-99
pct_tl_closed_L12M ---- 4.1384568281483056e-49
Tot_Missed_Pmnt ---- 9.572404615942641e-210
CC_TL ---- 5.710261952999025e-57
Home_TL ---- 0.0
PL_TL ---- 7.991892389241125e-235
Secured_TL ---- 0.0
Unsecured_TL ---- 2.102384431982427e-118
Other_TL ---- 0.0
Age_Oldest_TL ---- 0.0
Age_Newest_TL ---- 1.0465965445912546e-88
time_since_recent_payment ---- 8.943912807416738e-81
max_recent_level_of_deliq ---- 6.699932727692095e-81
num_deliq_6_12mts ---- 2.0772229883600566e-59
num_times_60p_dpd ---- 3.3833576401764997e-59
num_std_12mts ---- 0.0
num_sub ---- 1.6985545450345584e-22
num_sub_6mts ---- 0.016579119048249723
num_sub_12mts ---- 0.001299484021242129
num_dbt ---- 3.135918708089289e-11
num_dbt_12mts ---- 0.005446087209172642
num_lss ---- 0.009340555562870471
num_lss_12mts ---- 0.354899152686773
recent_level_of_deliq ---- 1.150955072948535e-97
CC_enq_L12m ---- 0.0
PL_enq_L12m -

In [ ]:
len(columns_to_be_kept_numerical)

In [ ]:
categorical_columns

In [ ]:
features = columns_to_be_kept_numerical + categorical_columns

In [ ]:
df = df[features]

In [ ]:
df.head()

In [ ]:
df.shape

In [17]:
class EducationMapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.education_mapping = {
            'SSC': 1,
            '12TH': 2,
            'GRADUATE': 3,
            'UNDER GRADUATE': 3,
            'POST-GRADUATE': 4,
            'OTHERS': 1,
            'PROFESSIONAL': 3
        }

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X['EDUCATION'] = X['EDUCATION'].map(self.education_mapping)
        return X

# Define the OneHotEncoder for the categorical columns
ohe = OneHotEncoder(sparse_output=False, drop='first')

# Columns to apply OneHotEncoder to
cat_columns = ['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

# Apply OneHotEncoder via ColumnTransformer
column_transformer = ColumnTransformer(
    transformers=[
        ('education_mapper', EducationMapper(), ['EDUCATION']),
        ('onehot', ohe, cat_columns),
        ('num', 'passthrough', columns_to_be_kept_numerical)  # Pass through numerical columns unchanged
    ]
)

# Separately encode the target column
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['Approved_Flag'])

# Split the dataset into features and target
X = df.drop(columns=['Approved_Flag'],axis = 1)

# Fit the transformer and transform the data
X_transformed = column_transformer.fit_transform(X)

# Convert the result back to a DataFrame with appropriate column names
# Get the feature names created by the OneHotEncoder
encoded_columns = column_transformer.named_transformers_['onehot'].get_feature_names_out(cat_columns)

# Combine the encoded categorical columns and numerical column names
all_columns = list(encoded_columns) + columns_to_be_kept_numerical + ['EDUCATION']

# Convert the transformed data back into a DataFrame
X_encoded = pd.DataFrame(X_transformed, columns=all_columns)

# Display the resulting encoded DataFrame
X_encoded.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 50 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   MARITALSTATUS_Single          42064 non-null  float64
 1   GENDER_M                      42064 non-null  float64
 2   last_prod_enq2_CC             42064 non-null  float64
 3   last_prod_enq2_ConsumerLoan   42064 non-null  float64
 4   last_prod_enq2_HL             42064 non-null  float64
 5   last_prod_enq2_PL             42064 non-null  float64
 6   last_prod_enq2_others         42064 non-null  float64
 7   first_prod_enq2_CC            42064 non-null  float64
 8   first_prod_enq2_ConsumerLoan  42064 non-null  float64
 9   first_prod_enq2_HL            42064 non-null  float64
 10  first_prod_enq2_PL            42064 non-null  float64
 11  first_prod_enq2_others        42064 non-null  float64
 12  pct_tl_open_L6M               42064 non-null  float64
 13  p

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

In [19]:
model = xgb.XGBClassifier(objective='multi:softmax',
                                       num_class=4,
                                       colsample_bytree = 0.5,
                                       learning_rate = 1.0,
                                       max_depth = 3,
                                       alpha = 10,
                                       n_estimators = 100,random_state = 42)

In [20]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)

0.7802210864138833

In [ ]:
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('column_transformer.pkl', 'wb') as f:
    pickle.dump(column_transformer, f)

In [ ]:
with open('columns_to_be_kept_numerical.pkl', 'wb') as f:
    pickle.dump(columns_to_be_kept_numerical, f)

In [ ]:
df_transformed = column_transformer.transform(df)

# Convert the result back to a DataFrame with appropriate column names
# Get the feature names created by the OneHotEncoder
encoded_columns = column_transformer.named_transformers_['onehot'].get_feature_names_out(cat_columns)

# Combine the encoded categorical columns and numerical column names
all_columns = list(encoded_columns) + columns_to_be_kept_numerical + ['EDUCATION']

# Convert the transformed data back into a DataFrame
X_encoded = pd.DataFrame(df_transformed, columns=all_columns)

In [ ]:
predictions = model.predict(X_encoded)

# Add predictions as a new column to the original dataframe
output_df = df_unseen.copy()
output_df['target_variable'] = predictions

In [ ]:
output_df.to_excel('/content/drive/MyDrive/Colab Notebooks/credit risk modelling/model_predictions_.xlsx', index = False)

In [ ]:
output_df['target_variable'].value_counts()

,count
target_variable,
1,68
3,12
2,11
0,9


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define the XGBClassifier with the initial set of hyperparameters
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

# Define the parameter grid for hyperparameter tuning

param_grid = {
  'colsample_bytree': [0.1, 0.3, 0.5, 0.7, 0.9],
  'learning_rate'   : [0.001, 0.01, 0.1, 1],
  'max_depth'       : [3, 5, 8, 10],
  'alpha'           : [1, 10, 100],
  'n_estimators'    : [10,50,100]
}

In [ ]:
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model with the best hyperparameters on the test set
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test, y_test)
print("Test Accuracy:", accuracy)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Hyperparameters: {'alpha': 1, 'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Test Accuracy: 0.7764174491857839
